In [9]:
import json
import openpyxl

# Load the workbook
wb = openpyxl.load_workbook('shop_site.xlsx')

comments_of_shops = {}
# Iterate over each sheet
for sheet_name in wb.sheetnames:
    if sheet_name == 'Sheet1':
        continue
    # Select the sheet
    sheet = wb[sheet_name]

    comments = []

    # Get the row values (assuming first row is title)
    titles = [cell.value for cell in sheet[1]]

    # skip if empty
    if not titles:
        continue

    # Iterate over each row starting from 2nd row (assuming first row is title)
    for row_num in range(2, sheet.max_row + 1):
        comment = {}
        # Get the row values
        row_values = [cell.value for cell in sheet[row_num]]
        for i in range(len(titles)):
            comment[titles[i]] = row_values[i]
        comments.append(comment)
    comments_of_shops[sheet_name] = comments

with open('comments_of_shops.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))


In [13]:
import json
# print(open('comments_of_shops.json','r').read())
comments_of_shops = json.loads(open('comments_of_shops.json','rb').read())
comments_of_shops_with_review = json.loads(open('comments_of_shops_with_review.json', 'rb').read())
for shop, comments in comments_of_shops.items():
    if comments_of_shops_with_review.get(shop):
        for index in range(len(comments)):
            comments[index].update(comments_of_shops_with_review[shop][index])
with open('comments_of_shops_with_review.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))
comments_of_shops_with_review.keys()

In [20]:
print(comments_of_shops['BRANDX·深蓝 第二季——沉浸式海洋数字艺术展'][0])

{'UserName': 'ohmynala', 'UserLink': 'https://www.dianping.com/member/1025560072', 'UserAvatar': 'https://p1.meituan.net/userheadpicbackend/121c15e14a1af390a38c473da9969323169644.jpg%4048w_48h_1e_1c_1l%7Cwatermark%3D0', 'Score': '', 'Content': '非常棒的体验 很棒的展，小孩子玩的很开心，适合带宝宝打卡，进去就有工作人员耐心的介绍和指导，里面还有很多互动的环节，做得非常细致入微，每一个环节都配有工作人员，🔛探索海洋生态！万物并存，生生不息，了解海洋生物和海底环境，保护环境保护地球🌍。', 'Images': 'https://qcloud.dpfile.com/pc/S638yXVTWEHaurk9bi69Xm5RL38og-6eAQQuvakw4aiMeRpyKCg0A0wXsXAOAdxojoJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg, https://qcloud.dpfile.com/pc/bo227U8b8dMR9kJY8FwnHTc5FcoWJSKYqxnDeMKG6pBiWRr1Oyx9E1Ltd1LLhGf7joJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg, https://qcloud.dpfile.com/pc/9piAYvVFD7w9_ODyGOFSZ1FEyiR4lQSTh_06hNx1tfsM_mJet2qLpG8XZRY-E7rljoJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg', 'DateTime': '2024-07-15 10:25', 'ShopName': 'BRANDX·深蓝 第二季——沉浸式海洋

In [14]:
import re
from tqdm import tqdm
from ollama_chat import chat_reponse
llama3 = 'llama3.1'
gemma2_q3 = 'gemma2:27b-instruct-q3_K_L'
gemma2 = 'gemma2:9b-instruct-q6_K'
gemma2_b = 'gemma2:27b-instruct-q5_K_M'
qwen2 = 'qwen2:7b-instruct-q6_K'

keys = ['value', 'immersive', 'guidance', 'followup', 'content']
# a regex string that matches the first five 1 digit numbers seperated by spaces to these keys: value, immersive, guidance, followup, and content. Then all other charactors that follows
review_re = r'(?P<value>\d)\s(?P<immersive>\d)\s(?P<guidance>\d)\s(?P<followup>\d)\s(?P<content>\d)(?P<reason>.*)'

for shop in comments_of_shops:
    print(shop)
    comments = comments_of_shops[shop]
    for comment in tqdm(comments):
        if comment.get('reason'):
            continue
        text = comment.get('Content')
        if not text:
            continue
        message = \
        """用户将输入来自某沉浸式艺术展的用户评论，请完整地阅读评论并对它进行分析。
        ###
        按照下面的五个方面对他进行打分：
        1. 性价比不高：消费者对于性价比有负面评价，这种评价来源于期望与实际体验之间的不匹配
        2. 沉浸体验很差：消费者对于付费进行沉浸体验本身的轻视、不尊重或不认可
        3. 体验引导不清晰：消费者常因体验的缺乏引导性而产生迷失感，对整体体验感到不满
        4. 后续消费环节/后续服务不充足：消费者在体验结束后仍对体验持续性有更高期待，认为缺少后续环节如周边，纪念品，收集品，信息交流渠道等
        5. 展览内容的负面评价：消费者对于沉浸展览的主题内容有负面评价，认为主题内容难以接受，例如太抽象、无法理解、看不懂等
        
        ###
        每一方面总分为5分，只能打整数分。
        评分时完全基于用户评价的文字内容，每个分数都要有自己参考的评论内容。
        只有评论中出现这一方面的负面评论时，才进行扣分。评价越消极扣分越多。
        分数越低说明这个评价包含的评价越符合这一个方面的描述。

        ###
        如果评论中没有提及某一个方面的相关信息，则这个方面评满分。
        如果需要推测评论者在这个方面的态度，则直接评满分。
        不要进行任何的推测，如果没有相关的信息直接评满分。

        ###
        在进行回复时，请严格按照以下格式：
        <分数一> <分数二> <分数三> <分数四> <分数五>
        <评价理由>

        ###
        不要在回复中添加任何的前缀！
        评分理由中对每一个分数给出分别给出理由。
        """
        review = chat_reponse(text, model=gemma2_q3, system_prompt=message)
        # print(text)
        # print(review['message']['content'])
        try:
            match_dict = re.match(review_re, review['message']['content'], re.DOTALL).groupdict()
        except AttributeError:
            print(review['message']['content'])
            continue
        for key, value in match_dict.items():
            if key == 'reason':
                value = value.strip()
            comment[key] = value

with open('comments_of_shops_with_review.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))

再现山海经·大型沉浸式体验馆


 50%|█████     | 1/2 [00:22<00:22, 22.57s/it]


KeyboardInterrupt: 